In [3]:
from __future__ import print_function
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from math import sqrt
import sklearn
from sklearn.linear_model import LogisticRegression
import pickle
import time
import torch

from utils import test_accuracy, projection, veccos, binary_search_cx, \
                  find_exp_score, find_slab_score, project_l2_centroid, \
                  project_l2_centroid_straight, project_slab, project_slab_straight,\
                  contaminate_dataset, find_regime_threshold

from attackers import StraightAttack, SemiOnlineAttack, ConcentratedAttack, GreedyAttack

In [4]:
def experiment(dataset, taus=None, contamination_levels=[0]):
    
    filepath = "./data/"+dataset+"/"
    clf = LogisticRegression(fit_intercept=False, solver='liblinear')
    n_taus = 11
    res = [[0 for a in range(n_taus)] for b in contamination_levels]
    w_res = [[] for b in contamination_levels]
    acc_res = [[0 for a in range(n_taus)] for b in contamination_levels]
    
    for i in range(n_exp):    
        print ("Running the {}-th experiment".format(i))
        start_time = time.time()
        filename = filepath+str(i)
        with open(filename,"rb") as f:
            datasets = pickle.load(f)

        X_init, Y_init = datasets[0] # generate defense constraints and init w if necessary.
        X_clean, Y_clean = datasets[1] # the clean data stream
        X_valid, Y_valid = datasets[2] # validation set
        X_test, Y_test = datasets[3]   # the actual test set

        clf.fit(X_init, Y_init)
        w_0 = np.zeros((1,d))
        print ("Initial accuracy is {}".format(test_accuracy(X_test, Y_test, w_0)))
        w_t = -clf.coef_
        print ("Target accuracy is {}".format(test_accuracy(X_test, Y_test, w_t)))
       
        defense_display_names = {"norm":"L_2-norm",
                                 "L2": "L_2-distance-to-centroid",
                                 "slab":"Slab"}
        print ("Attack method : {} Defense method : {}".format(attack_method,
                                                              defense_display_names[defense_method]))
        if attack_method == "simplistic":
            attacker = StraightAttack()
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack) 
        elif attack_method == "greedy":
            attacker = GreedyAttack()
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack) 
        elif attack_method == "concentrated":
            attacker = ConcentratedAttack()
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack)         
        elif attack_method == "semi-online":
            attacker = SemiOnlineAttack()
            X_adv, Y_adv = X_clean[:n_attack, :], Y_clean[:n_attack]
            attacker.set_param(datasets, w_0, w_t, R, eta, 
                                        defense, n_iter_warmup, n_attack, (-X_adv, Y_adv))
        attacker.set_init_set(X_init, Y_init)
        
        if defense_method == "slab":
            mu, scores = attacker.slab_scores()
        elif defense_method == "norm":
            mu, scores = attacker.l2_norms()
        elif defense_method == "L2":
            mu, scores = attacker.l2_distances_to_centroid()
        
        tau_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
        taus = [0]+[scores[int(tau_level*len(scores))-1] for tau_level in tau_levels]
        print ("taus to be tested are {}".format(taus))
        
        for j,contamination_level in enumerate(contamination_levels):
            
            X_init_t, Y_init_t = contaminate_dataset((np.copy(X_init), Y_init),contamination_level, R)
            attacker.set_init_set(X_init, Y_init)
            
            for k,tau in enumerate(taus):
                attacker.set_defense_threshold(defense_method, tau)
                attacker.set_defense_range(defense_range)
                attacker.set_init_set(X_init_t, Y_init_t)
                attacker.w_curr = w_0
                attacker.warmup(n_iter_warmup)
                w_b = attacker.w_curr
                attacker.set_target(-w_b)
                attacker.slab_scores()
                regime_thres = find_regime_threshold(mu, w_b, -w_b, defense_method)
                res_acc, _, res_w = attacker.attack()
                cos_sim = np.dot(w_b.flatten(), 
                                 res_w.flatten())/(np.linalg.norm(w_b)*np.linalg.norm(res_w)+1e-5) 
                print ("cosine score: {} test acc: {}".format(round(cos_sim,3), 
                                                              min(res_acc)))
                res[j][k] += (cos_sim/n_exp)
                acc_res[j][k] += (min(res_acc)/n_exp)
                w_res[j].append(res_w)
                attacker.reset()
        print("--- %s seconds ---" % (time.time() - start_time))

    print ("Predicted threshold of regimes: {}".format(regime_thres))
    results = [res, w_res, acc_res, taus, contamination_levels, regime_thres]
    filepath = "_".join(["./results/", dataset, attack_method, defense_method, defense_range])
      
    print (filepath)
    with open(filepath, "wb") as f:
        pickle.dump(results, f)
    filepath = "_".join(["./results/", dataset, attack_method, defense_method, defense_range])
    print (filepath)    
    with open(filepath, "rb") as f:
        results = pickle.load(f)
    res, w_res, acc_res, taus, contamination_levels, regime_thres = results

In [7]:
n_exp = 1
d, eta = 784, 0.01
n_attack, n_clean, n_init, n_test, n_valid = 100, 8000, 1000, 1000, 500
n_iter_warmup = n_clean
dataset = "MNIST"
R = 10
attack_methods = ["simplistic", "semi-online", "greedy",  "concentrated"]


In [8]:
defense_method = "norm"
defense = {defense_method:0}
for attack_method in attack_methods:
    defense_range = "att-only"
    experiment(dataset)

Running the 0-th experiment
Initial accuracy is 0.5
Target accuracy is 0.008
Attack method : simplistic Defense method : L_2-norm
taus to be tested are [0, 4.772877, 5.0128736, 5.2488356, 5.5068135, 5.786202, 6.0919604, 6.463639, 6.884351, 7.5627246, 10.939077]
cosine score: 1.0 test acc: 0.986
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
--- 8.562963724136353 seconds ---
Predicted threshold of regimes: (0, 0)
./results/_MNIST_simplistic_norm_att-only
./results/_MNIST_simplistic_norm_att-only
Running the 0-th experiment
Initial accuracy is 0.5
Target accuracy is 0.008
Attack method : semi-online Defense method : L_2-norm
taus to be tested are [0, 4.772877, 5.0128736, 5.2488356, 5.506813

In [9]:
defense_method = "L2"
defense = {defense_method:0}

for attack_method in attack_methods:
    defense_range = "att-only"
    experiment(dataset)

Running the 0-th experiment
Initial accuracy is 0.5
Target accuracy is 0.008
Attack method : simplistic Defense method : L_2-distance-to-centroid
taus to be tested are [0, 3.8123448, 4.1796293, 4.5975723, 4.9293513, 5.237112, 5.542228, 5.9059076, 6.3425975, 6.980779, 10.630243]
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
cosine score: -1.0 test acc: 0.014
--- 13.129324674606323 seconds ---
Predicted threshold of regimes: (2.028869063172743, 2.6215456)
./results/_MNIST_simplistic_L2_att-only
./results/_MNIST_simplistic_L2_att-only
Running the 0-th experiment
Initial accuracy is 0.5
Target accuracy is 0.008
Attack method : semi-online Defense method : L_2-distance-to-centroid
taus to be tested

In [10]:
defense_method = "slab"
defense = {defense_method:0}

for attack_method in attack_methods:
    defense_range = "att-only"
    experiment(dataset)

Running the 0-th experiment
Initial accuracy is 0.5
Target accuracy is 0.008
Attack method : simplistic Defense method : Slab
taus to be tested are [0, 0.68383235, 1.3052802, 2.0000656, 2.7429845, 3.6035678, 4.5674076, 5.7759647, 7.519409, 10.817202, 29.806637]
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: 1.0 test acc: 0.986
cosine score: -1.0 test acc: 0.014
--- 8.307719707489014 seconds ---
Predicted threshold of regimes: (13.745005, 13.745005)
./results/_MNIST_simplistic_slab_att-only
./results/_MNIST_simplistic_slab_att-only
Running the 0-th experiment
Initial accuracy is 0.5
Target accuracy is 0.008
Attack method : semi-online Defense method : Slab
taus to be tested are [0, 0.68383235, 1.3052802, 2.0000656, 2.7